In [ ]:
import os

In [ ]:
os.getcwd()

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.distributions import Normal
from torch.utils.data import Subset
from torch.distributions import Categorical, Normal, StudentT
from torch.optim import SGD
from torch.optim.lr_scheduler import PolynomialLR
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as tr
import torchmetrics
from torchmetrics.functional import calibration_error
import math
import matplotlib.pyplot as plt
import random
from collections import deque, OrderedDict
from tqdm import trange
import tqdm
import copy
import typing
from typing import Sequence, Optional, Callable, Tuple, Dict, Union
import pandas as pd
from sklearn.metrics import roc_auc_score

from data import Data
from priors import *
from Networks import *
from BayesianNN import BNN_MCMC
from SGLD import SGLD

In [13]:
# Load MNIST dataset with specified transforms
# possible transforms: RandomRotation, RandomCrop, GaussianBlur
# avoid Normalize and ToTensor (already done), RandomHorizontalFlip (for MNIST), RandomVerticalFlip (for MNIST)
augmentations = tr.Compose([tr.RandomRotation(15)])

train_data, test_data = Data("MNIST", augmentations = None).get_data(num_train_samples=6000)

# Print some information about the dataset
print("Train data size: ", len(train_data))
print("Test data size: ", len(test_data))


Train data size:  6000
Test data size:  10000


In [20]:
prior = Isotropic_Gaussian()
lol = BNN_MCMC(
    train_data, 
    network = FullyConnectedNN(), 
    prior=prior, 
    num_epochs = 30, 
    max_size = 15, 
    burn_in = 5, 
    lr = 1e-3, 
    sample_interval = 2, 
    Temperature = 10)

lol.train()

Training Model


100%|██████████| 30/30 [00:13<00:00,  2.28it/s, acc=0.828, log_prior_normalized=8.53e+4, loss=8.53e+4, lr=0.000183, nll_loss=24.7]


In [21]:
acc = lol.test_accuracy(test_data)
ece = lol.test_calibration(test_data)
auroc = lol.test_auroc(test_data)
print("Test accuracy: ", acc)
print("Test ECE: ", ece)
print("Test AUROC: ", auroc)



Test accuracy:  0.8737
Test ECE:  0.10882765
Test AUROC:  0.986883858255555


In [ ]:
lol.get_metrics(test_data)

In [ ]:
# compute AUROC
x_test = test_data[:][0]
y_test = test_data[:][1]    

# get prediction probabilities
probs = lol.predict_probabilities(x_test)

# get argmax of predictions
preds = probs.argmax(dim=1)

# compute AUROC
auroc = roc_auc_score(y_test, probs, multi_class='ovo')

auroc




In [ ]:
# get weights from all models
param_flat_all = []
for model in lol.model_sequence:
    parameters = model.state_dict()
    param_values = list(parameters.values())
    param_flat = torch.cat([v.flatten() for v in param_values])
    param_flat_all.append(param_flat.flatten())

# put all weights in one array
params = np.concatenate(param_flat_all)

# plot weights
plt.hist(params, bins=1000, range=(-4, 4))
plt.title("Weights of all models, GaussianMixture(-2, 0.5, 2, 0.5) ")
plt.show()

In [ ]:
x, var = Normal_Inverse_Gamma(0, 5, 1, 1).sample(100000)
x = x.flatten().numpy()
var = var

# plot prior samples
plt.hist(x, bins=1000, range=(-5, 5))
plt.show()

# plot prior samples
print(var)


# Pretrain Prior

In [ ]:
# pretrain on FashionMNIST
pretrain_data, pretest_data = Data("FashionMNIST", augmentations = None).get_data(num_train_samples=600)

prior = Isotropic_Gaussian()
pretrainer = BNN_MCMC(pretrain_data, network = FullyConnectedNN(), prior=prior, num_epochs = 30, max_size = 15, burn_in = 5, lr = 1e-3, sample_interval = 2)
pretrainer.train()

# get pretrained prior parameters
mu, var = pretrainer.get_posterior_stats()



In [ ]:
pretrained_Isotropic_Gaussian = Isotropic_Gaussian(mu, var)


# Evaluation loop

In [ ]:
prior_list = [Isotropic_Gaussian(),
              StudentT_prior(),
              GaussianSpikeNSlab(),
              MixedLaplaceUniform(), 
              Normal_Inverse_Gamma()]

prior_list_v2 = [Isotropic_Gaussian,
              StudentT_prior,
              GaussianSpikeNSlab,
              MixedLaplaceUniform, 
              Normal_Inverse_Gamma]

sample_sizes = [3750, 7500, 15000, 30000, 60000, 120000]

Temperatures = [1, 0.001, 0.01, 0.1, 1, 10]


prior_list = [] 
for prior in prior_list_v2:
    for temp in Temperatures:
        prior_list.append(prior(Temperature=temp))
        
# prior list is then list of all priors initialized with the correpsonding temperature
# [prior1_temp1, prior2_temp1, ..., prior7_temp1, prior1_temp2, prior2_temp2, ... ]

In [ ]:
# preallocate pandas dataframe for results
results = pd.DataFrame(columns = ["Prior", "Sample Size", "Epochs", "Burn_in", "sample_int", "Temperature", "Test Accuracy", "Test ECE"], index = range(len(prior_list)*len(sample_sizes)))

results

In [ ]:
# run the experiment for all priors

# set seed for reproducibility
torch.manual_seed(0)

#create a dict for the different parameter values
base_epoch, base_burn_in, base_sample_interval, base_samplesize = 50, 10, 2, sample_sizes[-1]
args_dict = [(sample_size, (base_epoch*base_samplesize/sample_size, base_burn_in*base_samplesize/sample_size, base_sample_interval*base_samplesize/sample_size )) for sample_size in sample_sizes]
args_dict = dict(args_dict)

iteration = 1

for n in range(len(sample_sizes)):
    # get data
    if sample_sizes[n] == 120000:
        # if sample size is 120000, use data augmentation
        augmentations = tr.Compose([tr.RandomRotation(15)])
        train_data, test_data = Data("MNIST", augmentations = augmentations).get_data()
    else:
        # subsample original train data if sample size is smaller than 120000
        train_data, test_data = Data("MNIST", augmentations = None).get_data(num_train_samples=sample_sizes[n])
    
    for i in range(len(prior_list)):
        # get prior
        prior = prior_list[i]
        print(50*"-")
        print("Iteration: ", iteration, " of ", len(prior_list)*len(sample_sizes))
        iteration += 1
        print("Prior:       ", prior.name)
        print("Temperature: ", prior.Temperature.numpy())
        print("Sample size: ", sample_sizes[n])

        print("Epoch:       ", args_dict[sample_sizes[n]][0])
        print("Burn in:     ", args_dict[sample_sizes[n]][1])
        print("Sample interval: ", args_dict[sample_sizes[n]][2])

        # run BNN
        model = BNN_MCMC(
            train_data,
            network = FullyConnectedNN(),
            prior=prior,
            num_epochs = int(args_dict[sample_sizes[n]][0]),
            max_size = 10,
            burn_in = int(args_dict[sample_sizes[n]][1]),
            lr = 1e-3,
            sample_interval = int(args_dict[sample_sizes[n]][2]))

        model.train()

        # get test accuracy and ECE
        acc = model.test_accuracy(test_data)
        ece = model.test_calibration(test_data)

        print("Test accuracy: ", acc)
        print("Test ECE: ", ece)

        # save results
        results.iloc[i+n*len(prior_list), :] = prior.name, sample_sizes[n], args_dict[sample_sizes[n]][0], args_dict[sample_sizes[n]][1], args_dict[sample_sizes[n]][2], prior.Temperature.numpy(), acc, ece

        # save model
        #torch.save(model, "models/"+prior.name+"_"+str(sample_sizes[n])+"_"+str(prior.Temperature.numpy())+".pt")


In [ ]:
# print full pandas dataframe
results

In [ ]:
# load model 
model = torch.load("models/Isotropic Gaussian_3750_1.0.pt")

# get test accuracy and ECE
acc = model.test_accuracy(test_data)

# get test data
x_test = test_data[:][0]
y_test = test_data[:][1]

preds = model.predict_probabilities(x_test)
print(preds.shape)

ece = calibration_error(preds, y_test, n_bins = 10, task = "multiclass", norm="l1", num_classes=10)


print("Test accuracy: ", acc)
#print("Test ECE: ", ece)

In [ ]:
test_data[:][0].clone().detach()